In [211]:
import numpy as np
from math import gamma
import scipy.optimize as opt
import scipy.stats as sts
import matplotlib.pyplot as plt
from scipy.integrate import quad

## Problem 4.

In [275]:
c, k, w, r = np.loadtxt(r'C:\Users\Max\Documents\BootCamp2017\Econ\Wk4_StrEst\data\MacroSeries.txt', delimiter=',').T
lc = np.log(c)
lk = np.log(k)
lw = np.log(w)
lr = np.log(r)

In [17]:
def get_z(alpha, lr, lk):
#     lr = np.log(r)
#     lk = np.log(k)
    zi = lr - alpha - (alpha - 1) * lk
    
    return zi

def get_moms(params, c, k, w, r):
    alpha, beta, rho, mu = params
    
    zi = get_z(alpha, r, k)
    
    mom_1 = np.mean(zi[1:] - rho * zi[:-1] - (1 - rho) * mu)
    mom_2 = np.mean((zi[1:] - rho * zi[:-1] - (1 - rho) * mu) * zi[1:])
    
    
    mom_3 = np.mean(beta * alpha * np.exp(zi[1:]) * k[1:] ** (alpha - 1) *
                     (c[:-1] / c[1:])-1)
    
    mom_4 = np.mean((beta * alpha * np.exp(zi[1:]) * k[1:] ** (alpha - 1) *
                     (c[:-1] / c[1:])-1) * w[:-1])
    
#     mom_3 = np.mean(np.log(beta) + alpha + zi[1:] + (alpha - 1) * k[1:] +
#                    c[:-1] - c[1:])
#     mom_4 = np.mean((np.log(beta) + alpha  + zi[1:] + (alpha - 1) * k[1:] +
#                    c[:-1] - c[1:]) * w[:-1])
    return np.array([mom_1, mom_2, mom_3, mom_4])

def criterion(params, *args):
    alpha, beta, rho, mu = params
    c, k, w, r, W = args
    
    mom_args = c, k, w, r
    err_vec = get_moms(params, c, k, w, r)
    
    crit_val = np.dot(np.dot(err_vec.T, W), err_vec)
    
    return crit_val
    

In [18]:
alpha_0 = 0.4
beta_0 = 0.9
rho_0 = 0.8
mu_0 = 3.0
params_init = np.array([alpha_0, beta_0, rho_0, mu_0])
W = np.identity(4)
data_log = (lc, lk, lw, lr, W)

In [19]:
get_z(alpha_0, r, k)
get_moms(params_init, c, k, w, r)
criterion(params_init, c, k, w, r, W)

C:\Users\Max\Anaconda3\lib\site-packages\ipykernel\__main__.py:17: RuntimeWarning: overflow encountered in exp
C:\Users\Max\Anaconda3\lib\site-packages\ipykernel\__main__.py:20: RuntimeWarning: overflow encountered in exp


nan

In [20]:
params_init.shape

(4,)

In [21]:
res = opt.minimize(criterion, params_init, args=data_log,
                   method='L-BFGS-B',
                   bounds=((1e-10, 0.999999), (1e-10, 0.99999999), 
                          (-0.999999, 0.99999), (1e-10, None)))
res

      fun: 4.911741455783979e-06
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([  3.63039532e-03,  -1.39231679e-03,  -5.47066356e-04,
         2.14705911e-09])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 170
      nit: 28
   status: 0
  success: True
        x: array([  0.78458127,   0.16536035,   0.99989631,  32.07124817])

In [22]:
alpha, beta, rho, mu = res.x
crit = res.fun

print('Estimation results',
      '\n\nalpha:\t\t\t', alpha,
      '\nbeta:\t\t\t', beta,
      '\nrho:\t\t\t', rho,
      '\nmu :\t\t\t', mu,
     '\ncriterion :\t\t', crit)
print()

Estimation results 

alpha:			 0.784581267613 
beta:			 0.165360348825 
rho:			 0.999896310793 
mu :			 32.0712481696 
criterion :		 4.91174145578e-06

